This jupyter notebook will be used to prepare for raw data npz files to be fed into the model

In [1]:
import re
import os
import numpy as np
import math
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys

import gzip
import os
import re
import tarfile

from six.moves import urllib
import numpy as np
import math

from tensorflow.python.platform import gfile
import tensorflow as tf

In [2]:
pac_data_directory = "pac_data" #Directory of raw numpy images
data_directory = "data_2" # Directory that contain two subdirectories, '0' and '1'. Within
                          # each of these subdirectories, will be stored files containing filenames / paths that point
                          # to the raw npz images

In [3]:
potential_sensor_directories = os.listdir(pac_data_directory)
sensor_directories = [sensor_dir for sensor_dir in potential_sensor_directories if os.path.isdir(pac_data_directory + "/" + sensor_dir)]

When you print 'sensor_directories', you should see all sensor numbers that are listed under the directory of 'pac_data_directory'

In [4]:
print(sensor_directories)

['02', '04', '06', '08', '10', '11', '15', '21', '22', '23', '24', '39', '52', '59', '62', '63', '72']


In [5]:
sensor_directory_to_list_of_filenames_of_images_labeled_zero_dict = {}
sensor_directory_to_list_of_filenames_of_images_labeled_one_dict = {}
pattern = r"\d{8}_\d{6}_\d{3}.npz"
for sensor_dir in sensor_directories:
    if os.path.isdir(pac_data_directory + "/" + sensor_dir + "/0"):
        potential_filenames_of_images_labeled_with_zero = os.listdir(pac_data_directory + "/" + sensor_dir + "/0")
        filenames_of_images_labeled_with_zero = [pac_data_directory + "/" + sensor_dir + "/0/" + f for f in potential_filenames_of_images_labeled_with_zero if os.path.isfile(pac_data_directory + "/" + sensor_dir + "/0/" + f) and re.match(pattern, f)]
        sensor_directory_to_list_of_filenames_of_images_labeled_zero_dict[sensor_dir] = filenames_of_images_labeled_with_zero
    if os.path.isdir(pac_data_directory + "/" + sensor_dir + "/1"):
        potential_filenames_of_images_labeled_with_one =  os.listdir(pac_data_directory + "/" + sensor_dir + "/1")
        filenames_of_images_labeled_with_one = [pac_data_directory + "/" + sensor_dir + "/1/" + f for f in potential_filenames_of_images_labeled_with_one if os.path.isfile(pac_data_directory + "/" + sensor_dir + "/1/" + f) and re.match(pattern, f)]
        sensor_directory_to_list_of_filenames_of_images_labeled_one_dict[sensor_dir] = filenames_of_images_labeled_with_one
    

The below cell will be used to split the filenames or store all filenames under one file, separated by classtype.

In [6]:
split_for_training_dev_and_testing = True
custom_filename = "custom_filename"

In [7]:
for sensor_dir in sensor_directories:
    if os.path.isdir(pac_data_directory + "/" + sensor_dir + "/0"):
        if split_for_training_dev_and_testing:
            dataset_training_path_sensor = data_directory + "/0/training_filenames_" + sensor_dir + ".txt"
            dataset_dev_path_sensor = data_directory + "/0/dev_filenames_" + sensor_dir + ".txt"
            dataset_testing_path_sensor = data_directory  + "/0/testing_filenames_" + sensor_dir + ".txt"
            numpy_examples_from_dataset_zero_labeled = np.array(sensor_directory_to_list_of_filenames_of_images_labeled_zero_dict[sensor_dir])
            random_ordered_numpy_examples_from_dataset_zero_labeled = numpy_examples_from_dataset_zero_labeled[np.random.permutation(len(sensor_directory_to_list_of_filenames_of_images_labeled_zero_dict[sensor_dir]))]
            num_of_examples_in_training_set_zero_labeled = math.ceil(len(numpy_examples_from_dataset_zero_labeled) * 0.70)
            num_of_examples_in_dev_set_zero_labeled = math.ceil(math.floor(len(numpy_examples_from_dataset_zero_labeled) * 0.30) * (2.0 / 3.0))
            num_of_examples_in_testing_set_zero_labeled = math.floor(math.floor(len(numpy_examples_from_dataset_zero_labeled) * 0.30) * (1.0/3.0))
            training_set_zero_labeled = random_ordered_numpy_examples_from_dataset_zero_labeled[:num_of_examples_in_training_set_zero_labeled]
            dev_set_zero_labeled = random_ordered_numpy_examples_from_dataset_zero_labeled[num_of_examples_in_training_set_zero_labeled:num_of_examples_in_dev_set_zero_labeled + num_of_examples_in_training_set_zero_labeled]
            testing_set_zero_labeled = random_ordered_numpy_examples_from_dataset_zero_labeled[num_of_examples_in_dev_set_zero_labeled + num_of_examples_in_training_set_zero_labeled:]
            with gfile.GFile(dataset_training_path_sensor, mode="a") as training_file:
                for filename_of_zero_labeled_image_for_training in training_set_zero_labeled:
                    training_file.write(filename_of_zero_labeled_image_for_training + "," + "0" + "\n")
            with gfile.GFile(dataset_dev_path_sensor, mode="a") as dev_file:
                for filename_of_zero_labeled_image_for_dev in dev_set_zero_labeled:
                    dev_file.write(filename_of_zero_labeled_image_for_dev + "," + "0" + "\n")
            with gfile.GFile(dataset_testing_path_sensor, mode="a") as testing_file:
                for filename_of_zero_labeled_image_for_testing in testing_set_zero_labeled:
                    testing_file.write(filename_of_zero_labeled_image_for_testing + "," + "0" + "\n")
        else:
            dataset_custom_path_sensor = data_directory + "/0/" + custom_filename + "_" + sensor_dir + ".txt"
            numpy_examples_from_dataset_zero_labeled = np.array(sensor_directory_to_list_of_filenames_of_images_labeled_zero_dict[sensor_dir])
            random_ordered_numpy_examples_from_dataset_zero_labeled = numpy_examples_from_dataset_zero_labeled[np.random.permutation(len(sensor_directory_to_list_of_filenames_of_images_labeled_zero_dict[sensor_dir]))]
            with gfile.GFile(dataset_custom_path_sensor, mode="a") as custom_file:
                for custom_image_filename in random_ordered_numpy_examples_from_dataset_zero_labeled:
                    custom_file.write(custom_image_filename + "," + "0" + "\n")
            
    if os.path.isdir(pac_data_directory + "/" + sensor_dir + "/1"):
        if split_for_training_dev_and_testing: 
            dataset_training_path_sensor = data_directory + "/1/training_filenames_" + sensor_dir + ".txt"
            dataset_dev_path_sensor = data_directory + "/1/dev_filenames_" + sensor_dir + ".txt"
            dataset_testing_path_sensor = data_directory + "/1/testing_filenames_" + sensor_dir + ".txt"
            numpy_examples_from_dataset_one_labeled = np.array(sensor_directory_to_list_of_filenames_of_images_labeled_one_dict[sensor_dir])
            random_ordered_numpy_examples_from_dataset_one_labeled = numpy_examples_from_dataset_one_labeled[np.random.permutation(len(sensor_directory_to_list_of_filenames_of_images_labeled_one_dict[sensor_dir]))]
            num_of_examples_in_training_set_one_labeled = math.ceil(len(numpy_examples_from_dataset_one_labeled) * 0.70)
            num_of_examples_in_dev_set_one_labeled = math.ceil(math.floor(len(numpy_examples_from_dataset_one_labeled) * 0.30) * (2.0 / 3.0))
            num_of_examples_in_testing_set_one_labeled = math.floor(math.floor(len(numpy_examples_from_dataset_one_labeled) * 0.30) * (1.0/3.0))
            training_set_one_labeled = random_ordered_numpy_examples_from_dataset_one_labeled[:num_of_examples_in_training_set_one_labeled]
            dev_set_one_labeled = random_ordered_numpy_examples_from_dataset_one_labeled[num_of_examples_in_training_set_one_labeled:num_of_examples_in_dev_set_one_labeled + num_of_examples_in_training_set_one_labeled]
            testing_set_one_labeled = random_ordered_numpy_examples_from_dataset_one_labeled[num_of_examples_in_dev_set_one_labeled + num_of_examples_in_training_set_one_labeled:]
            with gfile.GFile(dataset_training_path_sensor, mode="a") as training_file:
                for filename_of_one_labeled_image_for_training in training_set_one_labeled:
                    training_file.write(filename_of_one_labeled_image_for_training + "," + "1" +"\n")
            with gfile.GFile( dataset_dev_path_sensor, mode="a") as dev_file:
                for filename_of_one_labeled_image_for_dev in dev_set_one_labeled:
                    dev_file.write(filename_of_one_labeled_image_for_dev + "," + "1" + "\n")
            with gfile.GFile(dataset_testing_path_sensor, mode="a") as testing_file:
                for filename_of_one_labeled_image_for_testing in testing_set_one_labeled:
                    testing_file.write(filename_of_one_labeled_image_for_testing + "," + "1" + "\n")
        else:
            dataset_custom_path_sensor = data_directory + "/1/" + custom_filename + "_" + sensor_dir + ".txt"
            numpy_examples_from_dataset_one_labeled = np.array(sensor_directory_to_list_of_filenames_of_images_labeled_one_dict[sensor_dir])
            random_ordered_numpy_examples_from_dataset_one_labeled = numpy_examples_from_dataset_one_labeled[np.random.permutation(len(sensor_directory_to_list_of_filenames_of_images_labeled_one_dict[sensor_dir]))]
            with gfile.GFile(dataset_custom_path_sensor, mode="a") as custom_file:
                for custom_image_filename in random_ordered_numpy_examples_from_dataset_one_labeled:
                    custom_file.write(custom_image_filename + "," + "1" + "\n")